In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import dask.dataframe as dd
import os
import subprocess
import gzip


C:\Users\Deniz\AppData\Local\Temp\ipykernel_2144\707745631.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\Deniz\AppData\Local\Temp\ipykernel_2144\707745631.py:5: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning o

In [6]:
def process_date(directory_name):
    accumulator_df = None

    for filename in glob.glob(f"{directory_name}/18.csv.gz"):
        df = pd.read_csv(filename, sep='\t', compression='gzip', encoding='latin-1')
        if accumulator_df is None:
            accumulator_df = df
        else:
            accumulator_df = pd.concat([accumulator_df, df], ignore_index=True)
        del df # release memory

    # Now you can use accumulator_df
    return accumulator_df


In [7]:
mempool_day = process_date("data/20210423")
mempool_day
# test = mempool_day[mempool_day['status'] == 'confirmed']
# asdf = test.head(10)
# asdf

ParserError: Error tokenizing data. C error: Expected 27 fields in line 150432, saw 28


In [13]:
expected_num_fields = 27  # Adjust this based on the expected number of columns
file_path = 'data/20210423/18.csv.gz'  # Update this to the path of your gzip file
bad_lines_count = 0  # Initialize a counter for lines with an unexpected number of fields

with gzip.open(file_path, 'rt') as file:  # 'rt' mode for reading as text
    for i, line in enumerate(file):
        num_fields = len(line.split('\t'))  # Assuming a tab-separated file
        if num_fields != expected_num_fields:
            print(f"Line {i+1} has {num_fields} fields: {line}")
            bad_lines_count += 1  # Increment the counter for each bad line

print(f"Total number of lines with an unexpected number of fields: {bad_lines_count}")


UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 2359: character maps to <undefined>

In [6]:
private_transactions = mempool_day[mempool_day["timepending"] == 0]
print(private_transactions)

Empty DataFrame
Columns: [detecttime, hash, status, region, reorg, replace, curblocknumber, failurereason, blockspending, timepending, nonce, gas, gasprice, value, toaddress, fromaddress, input, network, type, maxpriorityfeepergas, maxfeepergas, basefeepergas, dropreason, rejectionreason, stuck, gasused, detect_date]
Index: []

[0 rows x 27 columns]


In [ ]:
#mempool = pd.read_csv('15.csv.gz', sep='\t', compression='gzip')
#mempool.columns = mempool.columns.str.strip()
#mempool.columns

target_directory = "20210602"
accumulator_df = None

for filename in glob.glob(f"{target_directory}/*.csv.gz"):
  df = pd.read_csv(filename, sep='\t', compression='gzip')
  if accumulator_df is None:
    accumulator_df = df
  else:
    accumulator_df = pd.concat([accumulator_df, df], ignore_index=True)
  del df # release memory

# Now you can use accumulator_df
mempool = accumulator_df

In [ ]:
private_transactions = mempool[mempool["timepending"] == 0]

# Print the selected DataFrame
print(private_transactions)

In [ ]:
%%bash
./download_slices.sh {20210602}

In [ ]:
def run_download_script(arguments):
  # Construct the command
  command = ["/bin/bash", "./download_slices.sh"] + arguments

  # Run the command with subprocess
  result = subprocess.run(command, capture_output=True)

  # Handle success or failure based on return code and output
  if result.returncode == 0:
    print("Download script completed successfully!")
  else:
    print("Error running download script:")
    print(result.stderr.decode())

In [ ]:
# Function to download and combine data
def download_and_combine_data(date):
  """Downloads data for the specified date, extracts, and combines into a DataFrame.

  Args:
    date: The date in YYYYMMDD format (e.g., "20240215").

  Returns:
    A pandas DataFrame containing the combined data.
  """

  # Create directory for downloaded files
  data_dir = f"{date}"
  os.makedirs(data_dir, exist_ok=True)  # Create directory safely

  run_download_script([date])


  # Check if downloaded files exist
  if not os.path.exists(data_dir):
    print(f"Data directory not found for date: {date}")
    return None

  accumulator_df = None
  # Combine all CSV.gz files into a single DataFrame
  for filename in glob.glob(os.path.join(data_dir, "*.csv.gz")):
    df = pd.read_csv(filename, sep='\t', compression='gzip')
    if accumulator_df is None:
      accumulator_df = df
    else:
      accumulator_df = pd.concat([accumulator_df, df], ignore_index=True)
    del df # release memory

  mempool = accumulator_df

  # Clean up extracted files (optional)
  # for filename in glob.glob(os.path.join(data_dir, "*.csv")):
  #   os.remove(filename)

  return mempool


In [ ]:
# Example usage
date = "20210602"
mempool = download_and_combine_data(date)

if mempool is not None:
  print("Successfully combined data into a DataFrame:")
  display(mempool.head())
else:
  print("Data download or processing failed.")

# private_transactions = mempool[mempool["timepending"] == 0]
# # Print the selected DataFrame
# print(private_transactions)
# print("daamn")

In [ ]:
mempool